# Load and Munge Song Lyric Data
Loads data, process, and munge for follow-on tasks.

### Imports
Common data manipulation and data viz toolsets plus regex.

In [2]:
# Data manipulation
import pandas as pd
import numpy as np

# ... and regex
import regex as re

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Prevent future/deprecation warnings from showing in output
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# Project file imports, exports
import os
# Create an output folder.
# [REF:How can I safely create a nested directory](https://stackoverflow.com/a/14364249)
#   > `pathlib.Path.mkdir` as used above recursively creates the directory 
#      and does not raise an exception if the directory already exists. 
#      If you don't need or want the parents to be created, skip the parents argument.
import pathlib
# Baseline
prj_home = pathlib.Path('..')
prj_home_data_dir = os.path.join(prj_home, 'lyrics/JSON')
prj_home_output_dir = os.path.join(prj_home, 'lyrics/data')

### Get Started with a Genius API-derived JSON file
- This also proved that the mis-selection of song "5-8-6" was correct in the fetched JSON file.

In [7]:
fact = '../lyrics/JSON/PowerCorruptionLies_4.json'

# Load
Artist = pd.read_json(fact, orient='index')

In [8]:
Artist.head(44)

,0
_type,song
album,"{'api_path': '/albums/58486', 'cover_art_url':..."
annotation_count,1
api_path,/songs/330886
apple_music_id,1040982145
apple_music_player_url,https://genius.com/songs/330886/apple_music_pl...
current_user_metadata,"{'permissions': ['follow', 'pyong', 'add_commu..."
custom_performances,[]
description,{'plain': '5-8-6 sounds like a previous versio...
description_annotation,"{'_type': 'referent', 'annotator_id': 1600528,..."


In [9]:
# Its the same thing as ^^, just as Series
Artist[0].to_dict

<bound method Series.to_dict of _type                                                                        song
album                           {'api_path': '/albums/58486', 'cover_art_url':...
annotation_count                                                                1
api_path                                                            /songs/330886
apple_music_id                                                         1040982145
apple_music_player_url          https://genius.com/songs/330886/apple_music_pl...
current_user_metadata           {'permissions': ['follow', 'pyong', 'add_commu...
custom_performances                                                            []
description                     {'plain': '5-8-6 sounds like a previous versio...
description_annotation          {'_type': 'referent', 'annotator_id': 1600528,...
embed_content                   <div id='rg_embed_link_330886' class='rg_embed...
featured_artists                                                  

In [10]:
Artist[0][1]

{'api_path': '/albums/58486',
 'cover_art_url': 'https://images.genius.com/b213883e0b31a7788959a1e96b5dc942.1000x1000x1.jpg',
 'full_title': 'Power, Corruption and Lies by New Order',
 'id': 58486,
 'name': 'Power, Corruption and Lies',
 'url': 'https://genius.com/albums/New-order/Power-corruption-and-lies',
 'artist': {'api_path': '/artists/37694',
  'header_image_url': 'https://images.genius.com/657eda0d0992cb31a2c3851fe6263b93.1000x563x1.jpg',
  'id': 37694,
  'image_url': 'https://images.genius.com/76e41832f0bc61e5e535ebc80a6cd9c9.519x519x1.jpg',
  'is_meme_verified': False,
  'is_verified': False,
  'name': 'New Order',
  'url': 'https://genius.com/artists/New-order'}}

In [11]:
#al_mkey = pd.Series(Artist[0]['album']['id'])
al_reldate = pd.Series(Artist[0]['release_date_for_display'])

from pandas.io.json import json_normalize
al_album = json_normalize(Artist[0]['album'])

In [12]:
#type(al_mkey)
#al_mkey.to_frame('id')
type(al_reldate)
al_reldate.to_frame('release_date')
al_album.head()

pandas.core.series.Series

,release_date
0,"May 1, 1983"


,api_path,cover_art_url,full_title,id,name,url,artist.api_path,artist.header_image_url,artist.id,artist.image_url,artist.is_meme_verified,artist.is_verified,artist.name,artist.url
0,/albums/58486,https://images.genius.com/b213883e0b31a7788959...,"Power, Corruption and Lies by New Order",58486,"Power, Corruption and Lies",https://genius.com/albums/New-order/Power-corr...,/artists/37694,https://images.genius.com/657eda0d0992cb31a2c3...,37694,https://images.genius.com/76e41832f0bc61e5e535...,False,False,New Order,https://genius.com/artists/New-order


In [13]:
al_rec = pd.concat([#al_mkey.to_frame('id'), 
                    al_reldate.to_frame('release_date'), 
                    al_album] , axis=1, join='inner')

In [14]:
al_rec.head()

,release_date,api_path,cover_art_url,full_title,id,name,url,artist.api_path,artist.header_image_url,artist.id,artist.image_url,artist.is_meme_verified,artist.is_verified,artist.name,artist.url
0,"May 1, 1983",/albums/58486,https://images.genius.com/b213883e0b31a7788959...,"Power, Corruption and Lies by New Order",58486,"Power, Corruption and Lies",https://genius.com/albums/New-order/Power-corr...,/artists/37694,https://images.genius.com/657eda0d0992cb31a2c3...,37694,https://images.genius.com/76e41832f0bc61e5e535...,False,False,New Order,https://genius.com/artists/New-order


In [15]:
al_rec.drop(['api_path', 
           #'full_title', 
           'artist.api_path', 
           'artist.header_image_url', 
           'artist.id', 
           'artist.image_url', 
           'artist.is_meme_verified',
           'artist.is_verified'], inplace = True, axis = 1)

In [16]:
al_rec.head()

,release_date,cover_art_url,full_title,id,name,url,artist.name,artist.url
0,"May 1, 1983",https://images.genius.com/b213883e0b31a7788959...,"Power, Corruption and Lies by New Order",58486,"Power, Corruption and Lies",https://genius.com/albums/New-order/Power-corr...,New Order,https://genius.com/artists/New-order


In [17]:
al_rec.rename(columns=lambda x: x.replace('.', '_'), inplace=True)

In [18]:
al_rec.head()

,release_date,cover_art_url,full_title,id,name,url,artist_name,artist_url
0,"May 1, 1983",https://images.genius.com/b213883e0b31a7788959...,"Power, Corruption and Lies by New Order",58486,"Power, Corruption and Lies",https://genius.com/albums/New-order/Power-corr...,New Order,https://genius.com/artists/New-order


In [19]:
al_rec["id"]

0    58486
Name: id, dtype: int64

In [20]:
al_rec.set_index("id", drop = True, inplace = True)

In [21]:
al_rec.head()

,release_date,cover_art_url,full_title,name,url,artist_name,artist_url
id,,,,,,,
58486,"May 1, 1983",https://images.genius.com/b213883e0b31a7788959...,"Power, Corruption and Lies by New Order","Power, Corruption and Lies",https://genius.com/albums/New-order/Power-corr...,New Order,https://genius.com/artists/New-order


In [22]:
type(al_rec)
al_rec.shape

pandas.core.frame.DataFrame

(1, 7)

---
### Get Header Record From Each Selected Album in Genius-derived API JSONs
- Begin with a scan of the JSON file repository (directory) and extract the header record for each album.
- This data will be loaded into the main SQLite table.

In [13]:
#import os
json_al = []
with os.scandir(prj_home_data_dir) as i:
    for entry in i:
        if entry.is_file():
            if entry.name.endswith('_1.json'):
                json_al.append(entry.name)

In [14]:
json_al

['Brotherhood_1.json',
 'Technique_1.json',
 'LostSirens_1.json',
 'PowerCorruptionLies_1.json',
 'WaitingForTheSirensCall_1.json',
 'UnknownPleasures_1.json',
 'GetReady_1.json',
 'Republic_1.json',
 'Closer_1.json',
 'LowLife_1.json',
 'Movement_1.json']

In [15]:
from pandas.io.json import json_normalize

for i in json_al: 
    print(f"Now reading in album: {i}")
    # Load
    Artist = pd.read_json(os.path.join(prj_home_data_dir, i), orient='index')

    al_reldate = pd.Series(Artist[0]['release_date_for_display'])
    al_album = json_normalize(Artist[0]['album'])
    
    al_rec = pd.concat([#al_mkey.to_frame('id'), 
                        al_reldate.to_frame('release_date'), 
                        al_album] , axis=1, join='inner')
    
    al_rec.set_index("id", drop = True, inplace = True)    

    al_rec.drop(['api_path', 
               #'full_title', 
               'artist.api_path', 
               'artist.header_image_url', 
               'artist.id', 
               'artist.image_url', 
               'artist.is_meme_verified',
               'artist.is_verified'], inplace = True, axis = 1)    

    al_rec.rename(columns=lambda x: x.replace('.', '_'), inplace=True)    
    
    prj_home_output_csv = os.path.join(prj_home_output_dir, 'no.csv')
    al_rec.to_csv(prj_home_output_csv, mode='a', sep='\t', encoding='utf-8', index=True, header=False)

Now reading in album: Brotherhood_1.json
Now reading in album: Technique_1.json
Now reading in album: LostSirens_1.json
Now reading in album: PowerCorruptionLies_1.json
Now reading in album: WaitingForTheSirensCall_1.json
Now reading in album: UnknownPleasures_1.json
Now reading in album: GetReady_1.json
Now reading in album: Republic_1.json
Now reading in album: Closer_1.json
Now reading in album: LowLife_1.json
Now reading in album: Movement_1.json


---
### Get Song Lyrics From Each Selected Album in Genius-derived JSONs
- Similar to above, but now get the song lyrics for each song in each album.
- This data will be loaded into the child SQLite table.

In [7]:
#import os
json_fi = []
with os.scandir(prj_home_data_dir) as i:
    for entry in i:
        if entry.is_file():
            if entry.name.endswith('.json'):
                json_fi.append(entry.name)

In [8]:
json_fi

['Closer_9.json',
 'Brotherhood_1.json',
 'LowLife_3.json',
 'Technique_1.json',
 'GetReady_6.json',
 'WaitingForTheSirensCall_7.json',
 'Closer_5.json',
 'Republic_6.json',
 'Movement_3.json',
 'UnknownPleasures_7.json',
 'GetReady_10.json',
 'UnknownPleasures_6.json',
 'Movement_2.json',
 'Republic_7.json',
 'LostSirens_1.json',
 'Closer_4.json',
 'WaitingForTheSirensCall_6.json',
 'GetReady_7.json',
 'PowerCorruptionLies_1.json',
 'LowLife_2.json',
 'Closer_8.json',
 'WaitingForTheSirensCall_1.json',
 'Closer_3.json',
 'LostSirens_6.json',
 'Movement_5.json',
 'UnknownPleasures_1.json',
 'Republic_10.json',
 'Brotherhood_7.json',
 'Technique_7.json',
 'LowLife_5.json',
 'PowerCorruptionLies_6.json',
 'GetReady_1.json',
 'PowerCorruptionLies_7.json',
 'Technique_6.json',
 'LowLife_4.json',
 'Brotherhood_6.json',
 'Republic_11.json',
 'LowLife_8.json',
 'Movement_4.json',
 'LostSirens_7.json',
 'Republic_1.json',
 'Closer_2.json',
 'PowerCorruptionLies_8.json',
 'UnknownPleasures_3.js

In [12]:
from pandas.io.json import json_normalize

# for i in json_fi[:1] (szf) Testing!
for i in json_fi: 
    print(f"Now reading in song: {i}")
    # Load
    Artist = pd.read_json(os.path.join(prj_home_data_dir, i), orient='index')
      
    # Collect data
    al_mkey = pd.Series(Artist[0]['album']['id'])  
    song_id = pd.Series(Artist[0]['id']) #, index=['id'])
    song_title = pd.Series(Artist[0]['title']) #, index=['song_title'])
    song_lyrics = pd.Series(Artist[0]['lyrics']) #, index=['song_lyrics'])
    #key_desc, val_desc = list(Artist[0]['description'].items())[0]
    song_desc = pd.Series(val_desc) #, index=['song_desc'])
    
    # build DataFrame
    song_rec = pd.concat([al_mkey.to_frame('fk_id'), song_id, song_title, song_lyrics, song_desc], axis=1, join='inner')
    # set Index
    song_rec.set_index("fk_id", drop = True, inplace = True)
    # make proper column names
    song_rec.rename(columns={0:'song_id', 1:'song_title', 2:'song_lyrics', 3:'song_desc'}, inplace=True)
    
    prj_home_output_csv = os.path.join(prj_home_output_dir, 'lyrics.csv')
    song_rec.to_csv(prj_home_output_csv, mode='a', sep='\t', encoding='utf-8', index=True, header=False)

Now reading in song: Closer_9.json
Now reading in song: Brotherhood_1.json
Now reading in song: LowLife_3.json
Now reading in song: Technique_1.json
Now reading in song: GetReady_6.json
Now reading in song: WaitingForTheSirensCall_7.json
Now reading in song: Closer_5.json
Now reading in song: Republic_6.json
Now reading in song: Movement_3.json
Now reading in song: UnknownPleasures_7.json
Now reading in song: GetReady_10.json
Now reading in song: UnknownPleasures_6.json
Now reading in song: Movement_2.json
Now reading in song: Republic_7.json
Now reading in song: LostSirens_1.json
Now reading in song: Closer_4.json
Now reading in song: WaitingForTheSirensCall_6.json
Now reading in song: GetReady_7.json
Now reading in song: PowerCorruptionLies_1.json
Now reading in song: LowLife_2.json
Now reading in song: Closer_8.json
Now reading in song: WaitingForTheSirensCall_1.json
Now reading in song: Closer_3.json
Now reading in song: LostSirens_6.json
Now reading in song: Movement_5.json
Now re